In [1]:
# Setup notebook
!pip install datasets transformers evaluate sentencepiece accelerate

import evaluate
import numpy as np
import pandas as pd

from copy import deepcopy
from datasets import load_dataset, load_from_disk
from sklearn.metrics import accuracy_score, f1_score, hamming_loss
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer

from google.colab import drive
drive.mount("/content/drive")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
Mounted at /content/drive


In [2]:
# Setup evaluation
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def compute_metrics(results, threshold=0.3, pipeline=False):
    preds, labels = results
    preds = sigmoid(preds)

    # Predict argmax regardless of value
    for pred in preds:
        pred[np.argmax(pred)] = 1

    # Binarization
    preds = (preds > threshold).astype(int)
    refs = labels.astype(int)

    # Compute
    a = hamming_loss(refs, preds)
    b = accuracy_score(refs, preds)
    c = f1_score(refs, preds, average="micro")

    return {"hamming_loss": a, "accuracy": b, "f1": c}

In [3]:
# Load and split dataset
dataset = load_from_disk("/content/drive/MyDrive/semeval_splits")
train_split = dataset["train"]
test_split = dataset["test"]
coref_split = dataset["coref"]

# Load base model
model_path = "microsoft/deberta-v3-base"
tokenizer = AutoTokenizer.from_pretrained(model_path)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [4]:
# Configure training parameters
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/semeval_models",
    overwrite_output_dir=True,
    learning_rate=0.0001,
    num_train_epochs=5,
    eval_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    report_to="none"
)

# Variation 1: Classify 22 fine-grain roles

Implementation Source: https://huggingface.co/blog/Valerii-Knowledgator/multi-label-classification

In [5]:
# V1: Preprocess and tokenize training data
classes = sorted([x for x in set(train_split["label1"]) if x])
label2id = {x:i for i, x in enumerate(classes)}
id2label = {i:x for x, i in label2id.items()}

def preprocess(sample):
    text = f"""[CLS]{sample["text"]}[SEP]{sample["mention"]}[SEP]"""
    labels = [sample["label1"], sample["label2"], sample["label3"]]
    labels = [label for label in labels if label]
    multihot = [0.0 for _ in range(len(classes))]

    for label in labels:
        id = label2id[label]
        multihot[id] = 1.0

    sample = tokenizer(text, truncation=True)
    sample["labels"] = multihot

    return sample

tokenized_train = train_split.map(preprocess)
tokenized_test = test_split.map(preprocess)
tokenized_coref = coref_split.map(preprocess)

In [6]:
# V1: Instantiate and train model
model = AutoModelForSequenceClassification.from_pretrained(
    model_path,
    num_labels=len(classes),
    id2label=id2label,
    label2id=label2id,
    hidden_dropout_prob=0.5,
    attention_probs_dropout_prob=0.5,
    problem_type="multi_label_classification"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

print(trainer.evaluate(eval_dataset=tokenized_test))
print(trainer.evaluate(eval_dataset=tokenized_coref))

pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-6-bbb7ac82a59c>:12: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


model.safetensors:   0%|          | 0.00/371M [00:00<?, ?B/s]

Epoch,Training Loss,Validation Loss,Hamming Loss,Accuracy,F1,Runtime,Samples Per Second,Steps Per Second
1,0.327000,0.206800,0.086298,0.086957,0.096552,3.271300,42.185000,5.502000
2,0.196600,0.199782,0.084321,0.101449,0.117241,3.228800,42.740000,5.575000
3,0.194100,0.201402,0.084321,0.101449,0.117241,3.225300,42.786000,5.581000
4,0.189300,0.202338,0.084321,0.101449,0.117241,3.220600,42.850000,5.589000
5,0.190500,0.202759,0.082345,0.115942,0.137931,3.291300,41.929000,5.469000


{'eval_loss': 0.2027592808008194, 'eval_hamming_loss': 0.08234519104084322, 'eval_accuracy': 0.11594202898550725, 'eval_f1': 0.13793103448275862, 'eval_runtime': 3.2711, 'eval_samples_per_second': 42.188, 'eval_steps_per_second': 5.503, 'epoch': 5.0}
{'eval_loss': 0.20270425081253052, 'eval_hamming_loss': 0.08234519104084322, 'eval_accuracy': 0.11594202898550725, 'eval_f1': 0.13793103448275862, 'eval_runtime': 3.2181, 'eval_samples_per_second': 42.882, 'eval_steps_per_second': 5.593, 'epoch': 5.0}


# Variation 2: Classify 25 types + fine-grain roles

In [7]:
# V2: Preprocess and tokenize training data
classes = set(train_split["label1"] + train_split["superlabel"])
classes = sorted([x for x in classes if x])
label2id = {x:i for i, x in enumerate(classes)}
id2label = {i:x for x, i in label2id.items()}

def preprocess(sample):
    text = f"""[CLS]{sample["text"]}[SEP]{sample["mention"]}[SEP]"""
    labels = [
        sample["superlabel"],
        sample["label1"],
        sample["label2"],
        sample["label3"]
    ]
    labels = [label for label in labels if label]
    multihot = [0.0 for _ in range(len(classes))]

    for label in labels:
        id = label2id[label]
        multihot[id] = 1.0

    sample = tokenizer(text, truncation=True)
    sample["labels"] = multihot

    return sample

tokenized_train = train_split.map(preprocess)
tokenized_test = test_split.map(preprocess)
tokenized_coref = coref_split.map(preprocess)

Map:   0%|          | 0/138 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [8]:
# V2: Instantiate and train model
model = AutoModelForSequenceClassification.from_pretrained(
    model_path,
    num_labels=len(classes),
    id2label=id2label,
    label2id=label2id,
    problem_type="multi_label_classification"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

print(trainer.evaluate(eval_dataset=tokenized_test))
print(trainer.evaluate(eval_dataset=tokenized_coref))

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-8-5ade66ba2fb7>:10: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Hamming Loss,Accuracy,F1,Runtime,Samples Per Second,Steps Per Second
1,0.277700,0.222952,0.067826,0.000000,0.453271,3.238500,42.612000,5.558000
2,0.224200,0.221978,0.067826,0.000000,0.453271,3.228900,42.739000,5.575000
3,0.221600,0.218579,0.067826,0.000000,0.453271,3.228500,42.744000,5.575000
4,0.216500,0.216569,0.067826,0.000000,0.453271,3.234000,42.672000,5.566000
5,0.212600,0.215861,0.067826,0.000000,0.453271,3.311900,41.668000,5.435000


{'eval_loss': 0.21586087346076965, 'eval_hamming_loss': 0.06782608695652174, 'eval_accuracy': 0.0, 'eval_f1': 0.4532710280373832, 'eval_runtime': 3.2661, 'eval_samples_per_second': 42.253, 'eval_steps_per_second': 5.511, 'epoch': 5.0}
{'eval_loss': 0.21585378050804138, 'eval_hamming_loss': 0.06782608695652174, 'eval_accuracy': 0.0, 'eval_f1': 0.4532710280373832, 'eval_runtime': 3.2401, 'eval_samples_per_second': 42.591, 'eval_steps_per_second': 5.555, 'epoch': 5.0}


# Variation 3: Classify 3 types -> Classify 6/12/4 fine-grain roles

In [9]:
# V3: Preprocess and tokenize training data
classes = sorted([x for x in set(train_split["superlabel"]) if x])
label2id = {x:i for i, x in enumerate(classes)}
id2label = {i:x for x, i in label2id.items()}

def preprocess(sample):
    text = f"""[CLS]{sample["text"]}[SEP]{sample["mention"]}[SEP]"""
    label = sample["superlabel"]
    onehot = [0.0 for i in range(len(classes))]
    id = label2id[label]
    onehot[id] = 1.0

    sample = tokenizer(text, truncation=True)
    sample["labels"] = onehot

    return sample

tokenized_train = train_split.map(preprocess)
tokenized_test = test_split.map(preprocess)
tokenized_coref = coref_split.map(preprocess)

In [10]:
# V3: Instantiate and train model
model = AutoModelForSequenceClassification.from_pretrained(
    model_path,
    num_labels=len(classes),
    id2label=id2label,
    label2id=label2id,
    problem_type="multi_label_classification"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

print(trainer.evaluate(eval_dataset=tokenized_test))
print(trainer.evaluate(eval_dataset=tokenized_coref))

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-10-461ce5235af2>:10: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Hamming Loss,Accuracy,F1,Runtime,Samples Per Second,Steps Per Second
1,0.515600,0.484701,0.198068,0.702899,0.702899,3.228000,42.751000,5.576000
2,0.492000,0.493777,0.198068,0.702899,0.702899,3.224700,42.794000,5.582000
3,0.493200,0.487896,0.198068,0.702899,0.702899,3.230700,42.715000,5.572000
4,0.494300,0.481178,0.198068,0.702899,0.702899,3.232800,42.687000,5.568000
5,0.491800,0.481105,0.198068,0.702899,0.702899,3.275500,42.132000,5.495000


{'eval_loss': 0.4811045229434967, 'eval_hamming_loss': 0.19806763285024154, 'eval_accuracy': 0.7028985507246377, 'eval_f1': 0.7028985507246377, 'eval_runtime': 3.2522, 'eval_samples_per_second': 42.433, 'eval_steps_per_second': 5.535, 'epoch': 5.0}
{'eval_loss': 0.481092631816864, 'eval_hamming_loss': 0.19806763285024154, 'eval_accuracy': 0.7028985507246377, 'eval_f1': 0.7028985507246377, 'eval_runtime': 3.2107, 'eval_samples_per_second': 42.982, 'eval_steps_per_second': 5.606, 'epoch': 5.0}


# Variation 3a: Classify 6 protagonist roles

In [11]:
# V3-pros: Preprocess and tokenize training data
filtered = train_split.filter(lambda x: x["superlabel"] == "Protagonist")
classes = sorted([x for x in set(filtered["label1"]) if x])
label2id = {x:i for i, x in enumerate(classes)}
id2label = {i:x for x, i in label2id.items()}

def preprocess(sample):
    text = f"""[CLS]{sample["text"]}[SEP]{sample["mention"]}[SEP]"""
    labels = [sample["label1"], sample["label2"], sample["label3"]]
    labels = [label for label in labels if label]
    multihot = [0.0 for _ in range(len(classes))]

    for label in labels:
        id = label2id[label]
        multihot[id] = 1.0

    sample = tokenizer(text, truncation=True)
    sample["labels"] = multihot

    return sample

tokenized_train = filtered.map(preprocess)
tokenized_test = test_split.filter(lambda x: x["superlabel"] == "Protagonist").map(preprocess)
tokenized_coref = coref_split.filter(lambda x: x["superlabel"] == "Protagonist").map(preprocess)

In [12]:
# V3-pros: Instantiate and train model
model = AutoModelForSequenceClassification.from_pretrained(
    model_path,
    num_labels=len(classes),
    id2label=id2label,
    label2id=label2id,
    problem_type="multi_label_classification"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

print(trainer.evaluate(eval_dataset=tokenized_test))
print(trainer.evaluate(eval_dataset=tokenized_coref))

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-12-f8020dc8e3a7>:10: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Hamming Loss,Accuracy,F1,Runtime,Samples Per Second,Steps Per Second
1,0.501700,0.460882,0.226667,0.360000,0.346154,0.611200,40.904000,6.545000
2,0.462500,0.459496,0.226667,0.360000,0.346154,0.618400,40.426000,6.468000
3,0.443900,0.456042,0.226667,0.360000,0.346154,0.617400,40.492000,6.479000
4,0.450400,0.455219,0.226667,0.360000,0.346154,0.608700,41.071000,6.571000
5,0.416900,0.453016,0.226667,0.360000,0.346154,0.642700,38.896000,6.223000


{'eval_loss': 0.45301613211631775, 'eval_hamming_loss': 0.22666666666666666, 'eval_accuracy': 0.36, 'eval_f1': 0.34615384615384615, 'eval_runtime': 0.6424, 'eval_samples_per_second': 38.916, 'eval_steps_per_second': 6.227, 'epoch': 5.0}
{'eval_loss': 0.4565366506576538, 'eval_hamming_loss': 0.22666666666666666, 'eval_accuracy': 0.36, 'eval_f1': 0.34615384615384615, 'eval_runtime': 0.6049, 'eval_samples_per_second': 41.33, 'eval_steps_per_second': 6.613, 'epoch': 5.0}


# Variation 3b: Classify 12 antagonist roles

In [13]:
# V3-ants: Preprocess and tokenize training data
filtered = train_split.filter(lambda x: x["superlabel"] == "Antagonist")
classes = sorted([x for x in set(filtered["label1"]) if x])
label2id = {x:i for i, x in enumerate(classes)}
id2label = {i:x for x, i in label2id.items()}

def preprocess(sample):
    text = f"""[CLS]{sample["text"]}[SEP]{sample["mention"]}[SEP]"""
    labels = [sample["label1"], sample["label2"], sample["label3"]]
    labels = [label for label in labels if label]
    multihot = [0.0 for _ in range(len(classes))]

    for label in labels:
        id = label2id[label]
        multihot[id] = 1.0

    sample = tokenizer(text, truncation=True)
    sample["labels"] = multihot

    return sample

tokenized_train = filtered.map(preprocess)
tokenized_test = test_split.filter(lambda x: x["superlabel"] == "Antagonist").map(preprocess)
tokenized_coref = coref_split.filter(lambda x: x["superlabel"] == "Antagonist").map(preprocess)

In [14]:
# V3-ants: Instantiate and train model
model = AutoModelForSequenceClassification.from_pretrained(
    model_path,
    num_labels=len(classes),
    id2label=id2label,
    label2id=label2id,
    problem_type="multi_label_classification"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

print(trainer.evaluate(eval_dataset=tokenized_test))
print(trainer.evaluate(eval_dataset=tokenized_coref))

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-14-13e8d25d1a8d>:10: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Hamming Loss,Accuracy,F1,Runtime,Samples Per Second,Steps Per Second
1,0.338100,0.296809,0.147766,0.144330,0.165049,2.260400,42.912000,5.751000
2,0.290600,0.297339,0.146048,0.154639,0.174757,2.252600,43.062000,5.771000
3,0.284100,0.288966,0.135739,0.216495,0.233010,2.251600,43.081000,5.774000
4,0.276700,0.282003,0.137457,0.206186,0.223301,2.256000,42.997000,5.762000
5,0.269600,0.280307,0.140893,0.206186,0.203883,2.292700,42.309000,5.670000


{'eval_loss': 0.28030669689178467, 'eval_hamming_loss': 0.140893470790378, 'eval_accuracy': 0.20618556701030927, 'eval_f1': 0.20388349514563106, 'eval_runtime': 2.2984, 'eval_samples_per_second': 42.204, 'eval_steps_per_second': 5.656, 'epoch': 5.0}
{'eval_loss': 0.28074267506599426, 'eval_hamming_loss': 0.140893470790378, 'eval_accuracy': 0.1958762886597938, 'eval_f1': 0.20388349514563106, 'eval_runtime': 2.2535, 'eval_samples_per_second': 43.044, 'eval_steps_per_second': 5.769, 'epoch': 5.0}


# Variation 3c: Classify 4 innocent roles

In [15]:
# V3-inno: Preprocess and tokenize training data
filtered = train_split.filter(lambda x: x["superlabel"] == "Innocent")
classes = sorted([x for x in set(filtered["label1"]) if x])
label2id = {x:i for i, x in enumerate(classes)}
id2label = {i:x for x, i in label2id.items()}

def preprocess(sample):
    text = f"""[CLS]{sample["text"]}[SEP]{sample["mention"]}[SEP]"""
    labels = [sample["label1"], sample["label2"], sample["label3"]]
    labels = [label for label in labels if label]
    multihot = [0.0 for _ in range(len(classes))]

    for label in labels:
        id = label2id[label]
        multihot[id] = 1.0

    sample = tokenizer(text, truncation=True)
    sample["labels"] = multihot

    return sample

tokenized_train = filtered.map(preprocess)
tokenized_test = test_split.filter(lambda x: x["superlabel"] == "Innocent").map(preprocess)
tokenized_coref = coref_split.filter(lambda x: x["superlabel"] == "Innocent").map(preprocess)

In [16]:
# V3-ant: Instantiate and train model
model = AutoModelForSequenceClassification.from_pretrained(
    model_path,
    num_labels=len(classes),
    id2label=id2label,
    label2id=label2id,
    problem_type="multi_label_classification"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

print(trainer.evaluate(eval_dataset=tokenized_test))
print(trainer.evaluate(eval_dataset=tokenized_coref))

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-16-4043ff63977f>:10: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Hamming Loss,Accuracy,F1,Runtime,Samples Per Second,Steps Per Second
1,0.481200,0.349086,0.125000,0.750000,0.750000,0.362200,44.173000,5.522000
2,0.364600,0.347005,0.125000,0.750000,0.750000,0.363500,44.015000,5.502000
3,0.363300,0.345989,0.125000,0.750000,0.750000,0.370200,43.218000,5.402000
4,0.366700,0.339137,0.125000,0.750000,0.750000,0.366400,43.664000,5.458000
5,0.350900,0.327150,0.125000,0.750000,0.750000,0.396800,40.325000,5.041000


{'eval_loss': 0.32714980840682983, 'eval_hamming_loss': 0.125, 'eval_accuracy': 0.75, 'eval_f1': 0.75, 'eval_runtime': 0.3967, 'eval_samples_per_second': 40.329, 'eval_steps_per_second': 5.041, 'epoch': 5.0}
{'eval_loss': 0.32740241289138794, 'eval_hamming_loss': 0.125, 'eval_accuracy': 0.75, 'eval_f1': 0.75, 'eval_runtime': 0.3704, 'eval_samples_per_second': 43.201, 'eval_steps_per_second': 5.4, 'epoch': 5.0}
